In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
from torch import Tensor
#!pip install einops
!pip install vit-pytorch
from torchvision.models import resnet18
from vit_pytorch.mobile_vit import MobileViT

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
from numpy import load
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')
# x_train = load('/content/drive/MyDrive/Colab_data/data_256/x_train.npy')
# y_train = load('/content/drive/MyDrive/Colab_data/data_256/y_train.npy')
# x_test = load('/content/drive/MyDrive/Colab_data/data_256/x_test.npy')
# y_test = load('/content/drive/MyDrive/Colab_data/data_256/y_test.npy')
images = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_train.npy')
labels = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_train.npy')

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# x_train = load('x_train.npy')
# y_train = load('y_train.npy')
# x_test = load('x_test.npy')
# y_test = load('y_test.npy')
x_train = x_train.reshape((-1,3,256,256))
x_test = x_test.reshape((-1,3,256,256))

Mounted at /content/drive


In [3]:

print(np.shape(x_train))

(3601, 3, 256, 256)


In [4]:
torch.cuda.is_available()

True

In [10]:



image_size = 256

# Define transformations
train_transform = transforms.Compose([
    #transforms.RandomResizedCrop(256),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    #transforms.Resize(256),
    # transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    #transforms.Resize(256),
    # transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Apply transformations to custom datasets
x_train = train_transform(x_train)
x_test = val_transform(x_test)

# Create DataLoader objects
batch_size = 128

dataset_train = TensorDataset( x_train, Tensor(y_train).long() )
dataset_test = TensorDataset( x_test, Tensor(y_test).long())

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)




In [11]:
# Initialize model, loss, and optimizer
num_epochs = 300

train_loss_rec = np.zeros(num_epochs)
train_acc_rec =np.zeros(num_epochs) 
test_acc_rec = np.zeros(num_epochs)
model  = MobileViT(
    image_size = (256, 256),
    dims = [96, 120, 144],
    channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
    num_classes = 2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=1e-4, weight_decay=2e-2)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    train_loss_rec[epoch] = running_loss/len(train_loader)
    # Evaluation
    model.eval()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()


        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    print(f"train_Accuracy: {100 * train_correct / train_total}%")
    print(f"test_Accuracy: {100 * test_correct / test_total}%")
    train_acc_rec[epoch] =train_correct / train_total 
    test_acc_rec[epoch] =test_correct / test_total

Epoch 1/300, Loss: 0.6844115113389904
train_Accuracy: 50.152735351291305%
test_Accuracy: 52.275249722530525%
Epoch 2/300, Loss: 0.6674558335337145
train_Accuracy: 62.454873646209386%
test_Accuracy: 58.71254162042175%
Epoch 3/300, Loss: 0.6330415462625438
train_Accuracy: 71.56345459594557%
test_Accuracy: 68.81243063263041%
Epoch 4/300, Loss: 0.540918984289827
train_Accuracy: 79.4501527353513%
test_Accuracy: 77.9134295227525%
Epoch 5/300, Loss: 0.4311315982506193
train_Accuracy: 84.97639544570953%
test_Accuracy: 80.2441731409545%
Epoch 6/300, Loss: 0.371886695253438
train_Accuracy: 85.44848653151902%
test_Accuracy: 80.79911209766925%
Epoch 7/300, Loss: 0.303638763468841
train_Accuracy: 85.69841710635934%
test_Accuracy: 79.13429522752497%
Epoch 8/300, Loss: 0.2808024189595518
train_Accuracy: 89.6417661760622%
test_Accuracy: 80.02219755826859%
Epoch 9/300, Loss: 0.21725706499198388
train_Accuracy: 95.27908914190503%
test_Accuracy: 84.68368479467259%
Epoch 10/300, Loss: 0.18158966354255018


In [ ]:
x_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_test.npy')
y_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_test.npy')

x_test = x_test.reshape((-1,3,256,256))
x_test = test_transform(x_test)
dataset_test = TensorDataset( x_test, Tensor(y_test).long())
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [ ]:
 # Evaluation
model.eval()

test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
    


    
test_acc =test_correct / test_total

In [ ]:
from numpy import save

save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_no_noise_train_loss_rec.npy', train_loss_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_no_noise_train_acc_rec.npy', train_acc_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_no_noise_val_acc_rec.npy', test_acc_rec)
save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/Mvit_no_noise_test_acc.npy', test_acc)